In [1]:
import pyscfad
from pyscfad import gto,dft,scf
import matplotlib.pyplot as plt
import equinox as eqx
import pyscf
# from pyscf import gto,dft,scf
import numpy as np
import jax.numpy as jnp
import scipy
from ase import Atoms
from ase.io import read
import xcquinox as xce
from functools import partial
from ase.units import Bohr
import os, optax, jax
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]="false"
os.environ["XLA_PYTHON_CLIENT_ALLOCATOR"]="platform"

/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [2]:
#develop function for reading in networks with assume directory structure
p = '/home/awills/Documents/Research/xcquinox_pt/pbe0/c_3_16_mgga'
ninput = 4
use = []

def loadnet_from_strucdir(path, ninput, use=[]):
    sp = path.split('/')
    if '.eqx' in sp[-1]:
        f = sp[-1]
        sdir = sp[-2]
        fullpath = True
    else:
        sdir = sp[-1]
        f = sorted([i for i in os.listdir(path) if '.eqx' in i], key = lambda x: int(x.split('.')[-1]))[-1]
        fullpath = False
    
    loadnet = path if fullpath else os.path.join(path, f)
    levels = {'gga': 2, 'mgga': 3, 'nl': 4}
    net_type, ndepth, nhidden, level = sdir.split('_')
    if level == 'gga':
        if net_type == 'x':
            use = use if use else [1]
            thisnet = xce.net.eX(n_input=ninput, n_hidden=int(nhidden), use=use, depth=int(ndepth), lob=1.804)
        elif net_type == 'c':
            use = use if use else [2]
            thisnet = xce.net.eC(n_input=ninput, n_hidden=int(nhidden), use=use, depth=int(ndepth), ueg_limit=True)
    elif level == 'mgga':
        if net_type == 'x':
            use = use if use else [1, 2]
            thisnet = xce.net.eX(n_input=ninput, n_hidden=int(nhidden), use=use, depth=int(ndepth), ueg_limit=True, lob=1.174)
        elif net_type == 'c':
            use = use if use else []
            thisnet = xce.net.eC(n_input=ninput, n_hidden=int(nhidden), use=use, depth=int(ndepth), ueg_limit=True)
    elif level == 'nl':
        if net_type == 'x':
            use = use if use else []
            thisnet = xce.net.eX(n_input=ninput, n_hidden=int(nhidden), use=use, depth=int(ndepth), ueg_limit=True, lob=1.174)
        elif net_type == 'c':
            use = use if use else []
            thisnet = xce.net.eC(n_input=ninput, n_hidden=int(nhidden), use=use, depth=int(ndepth), ueg_limit=True)
    
    thisnet = eqx.tree_deserialise_leaves(loadnet, thisnet)
    return thisnet, levels[level]

net, netlevel = loadnet_from_strucdir(p, 4)

In [3]:
netlevel

3

In [4]:
PRETRAIN_LEVEL = 'MGGA'

REFERENCE_XC = 'PBE0'

N_HIDDEN = 16
DEPTH = 3
if PRETRAIN_LEVEL == 'GGA':
    localx = xce.net.eX(n_input=1, n_hidden=N_HIDDEN, use=[1], depth=DEPTH, lob=1.804)
    localc = xce.net.eC(n_input=3, n_hidden=N_HIDDEN, use=[2], depth=DEPTH, ueg_limit=True)
elif PRETRAIN_LEVEL == 'MGGA':
    localx = xce.net.eX(n_input=2, n_hidden=N_HIDDEN, use=[1, 2], depth=DEPTH, ueg_limit=True, lob=1.174)
    localc = xce.net.eC(n_input=4, n_hidden=N_HIDDEN, depth=DEPTH, use=[], ueg_limit=True)
elif PRETRAIN_LEVEL == 'NONLOCAL':
    localx = xce.net.eX(n_input=18, n_hidden=N_HIDDEN, depth=DEPTH, ueg_limit=True, lob=1.174)
    localc = xce.net.eC(n_input=16, n_hidden=N_HIDDEN, depth=DEPTH, ueg_limit=True)

ptmgxp = '/home/awills/Documents/Research/xcquinox_pt/pbe0/x_3_16_mgga'
ptmgcp = '/home/awills/Documents/Research/xcquinox_pt/pbe0/c_3_16_mgga'

if PRETRAIN_LEVEL == 'MGGA':
    try:
        xcs = sorted([i for i in os.listdir(ptmgxp) if 'xc.eqx' in i], key=lambda x: int(x.split('.')[-1]))[-1]
        localx = eqx.tree_deserialise_leaves(os.path.join(ptmgxp, xcs), localx)
    except Exception as e:
        print(e)
        print('couldnt read in pt network to overwrite exchange')
    try:
        xcs = sorted([i for i in os.listdir(ptmgcp) if 'xc.eqx' in i], key=lambda x: int(x.split('.')[-1]))[-1]
        localc = eqx.tree_deserialise_leaves(os.path.join(ptmgcp, xcs), localc)
    except Exception as e:
        print(e)
        print('couldnt read in pt network to overwrite correlation')


xc = xce.xc.eXC(grid_models=[localx, localc], heg_mult=True, level= {'GGA':2, 'MGGA':3, 'NONLOCAL':4}[PRETRAIN_LEVEL],
               verbose=True)



In [5]:
try:
    trainms = read('/home/awills/Documents/Research2/torch_dpy/subset09_nf/subat_ref_corrected.traj', ':')
except:
    trainms = read('/home/awills/Documents/Research/torch_dpy/subset09_nf/subat_ref_corrected.traj', ':')
mfs = []
mols = []
energies = []
dms = []
ao_evals = []
gws = []
eris = []
mo_occs = []
hcs = []
vs = []
ts = []
ss = []
hologaps = []
ogds = []
for idx, at in enumerate(trainms[16:17]):
    name, mol = xce.utils.ase_atoms_to_mol(at, basis='def2tzvpd')
    mol.verbose=9
    mol.build()
    mols.append(mol)
    mf = dft.RKS(mol, xc='SCAN')
    # mf = scf.UHF(mol)
    mf.grids.level = 1
    e_tot = mf.kernel()
    mf.conv_tol = 1e-6
    mfs.append(mf)
    dm = mf.make_rdm1()
    ao_eval = jnp.array(mf._numint.eval_ao(mol, mf.grids.coords, deriv=2))
    energies.append(mf.get_veff().exc)
    dms.append(dm)
    ogds.append(dm.shape)
    ao_evals.append(ao_eval)
    gws.append(mf.grids.weights)
    ts.append(mol.intor('int1e_kin'))
    vs.append(mol.intor('int1e_nuc'))
    mo_occs.append(mf.mo_occ)
    hcs.append(mf.get_hcore())
    eris.append(mol.intor('int2e'))
    ss.append(jnp.linalg.inv(jnp.linalg.cholesky(mol.intor('int1e_ovlp'))))
    # hologaps.append(mf.mo_energy[mf.mo_occ == 0][0] - mf.mo_energy[mf.mo_occ > 1][-1])

System: uname_result(system='Linux', node='aegis', release='5.15.0-105-generic', version='#115~20.04.1-Ubuntu SMP Mon Apr 15 17:33:04 UTC 2024', machine='x86_64')  Threads 20
Python 3.10.14 (main, Mar 21 2024, 16:24:04) [GCC 11.2.0]
numpy 1.26.4  scipy 1.11.4
Date: Mon Apr 29 18:21:21 2024
PySCF version 2.3.0
PySCF path  /home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf

[CONFIG] ARGPARSE = False
[CONFIG] DEBUG = False
[CONFIG] MAX_MEMORY = 4000
[CONFIG] TMPDIR = .
[CONFIG] UNIT = angstrom
[CONFIG] VERBOSE = 3
[CONFIG] conf_file = /home/awills/.pyscf_conf.py
[CONFIG] pyscf_numpy_backend = jax
[CONFIG] pyscf_scipy_backend = jax
[CONFIG] pyscf_scipy_linalg_backend = pyscfad
[CONFIG] pyscfad = True
[CONFIG] pyscfad_ccsd_implicit_diff = True
[CONFIG] pyscfad_scf_implicit_diff = True
[INPUT] verbose = 9
[INPUT] max_memory = 4000 
[INPUT] num. atoms = 1
[INPUT] num. electrons = 1
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 1
[INPUT] symmetry False subgrou

/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1215: UserWarning: Function mol.dumps drops attribute coords because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1215: UserWarning: Function mol.dumps drops attribute exp because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1215: UserWarning: Function mol.dumps drops attribute ctr_coeff because it is not JSON-serializable
  warnings.warn(msg)


Drop grids 140
    CPU time for setting up grids      1.17 sec, wall time      0.27 sec
MGGA ni.block_loop; input ao.shape=(10, 2348, 9), weight.shape=(2348,), coords.shape=(2348, 3)
nelec by numeric integration = 0.9997672826941552
    CPU time for vxc      0.61 sec, wall time      0.31 sec
E1 = -0.4935841095402088  Ecoul = 0.3420619878675808  Exc = -0.2869383427870689
init E= -0.438460464459697
    CPU time for initialize scf      3.61 sec, wall time      0.75 sec

WARN: HOMO 2.30209063038258 == LUMO -0.206086726476509

  mo_energy =
[-0.20608673  0.17815618  0.17815618  0.17815618  0.32090151  1.59187323
  1.59187323  1.59187323  2.30209063]
MGGA ni.block_loop; input ao.shape=(10, 2348, 9), weight.shape=(2348,), coords.shape=(2348, 3)
nelec by numeric integration = 0.0
    CPU time for vxc      0.19 sec, wall time      0.02 sec
E1 = 0.0  Ecoul = 0.0  Exc = 0.0
cycle= 1 E= 0  delta_E= 0.438  |g|=    0  |ddm|= 0.451
    CPU time for cycle = 1      0.58 sec, wall time      0.20 sec
dii

/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscfad/_src/util.py:108: UserWarning: Not taking derivatives wrt the leaves in the node <class 'pyscfad.dft.rks.VXC'> as none of those was specified.
  warnings.warn(f'Not taking derivatives wrt the leaves in '


MGGA ni.block_loop; input ao.shape=(10, 2348, 9), weight.shape=(2348,), coords.shape=(2348, 3)
nelec by numeric integration = 0.0
    CPU time for vxc      0.14 sec, wall time      0.01 sec


In [6]:
mf.e_tot

Array(0., dtype=float64)

In [7]:
mf.mo_energy

Array([-0.49980983, -0.09269776, -0.09269776, -0.09269776,  0.02708109,
        1.20270742,  1.20270742,  1.20270742,  1.8981434 ], dtype=float64)

In [8]:
xc(dms[0], ao_evals[0], gws[0], mf=mf, coor=mf.grids.coords)

custom gw and coor present in eval_grid_models; shapes: gw=(2348,), coor=(2348, 3)
eval_grid_models initial nan summary:
zeta, rs, rs_a, rs_b, exc_a, exc_b, exc_ab
0, 0, 0, 0, 0, 0, 0
l_1, descr shape: (2348,)
NaNs in descr from self.l_1 = 0
self.level > 0; descr1 Nans = 0
self.level > 0; descr2 Nans = 0
get_descriptors -> self.level > 0
descr1.shape=(2348,), descr2.shape=(2348,), descr.shape=(2348, 2)
l_2, descr shape: (2348,)
NaNs in descr from self.l_2 = 0
self.level > 1; descr3 Nans = 0
get_descriptors -> self.level > 1
descr3.shape=(2348, 1), descr.shape=(2348, 3)
l_3, descr shape: (2348,)
NaNs in descr from self.l_3 = 0
self.level > 2; pre-log descr4 Nans = 0
descr4.min/max: 0.013888168840292215, 0.013888168840292215
self.level > 2; descr4 Nans = 0
get_descriptors -> self.level > 2
descr4.shape=(2348, 1), descr.shape=(2348, 4)
get_descriptors, not spin_scaling -> descr.shape=(2348, 4)
l_1, descr shape: (2348,)
NaNs in descr from self.l_1 = 0
l_1, descr shape: (2348,)
NaNs in desc

Array(-8.5722289e-10, dtype=float64)

In [9]:
mf.mo_coeff

array([[ 2.36845987e-01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -1.46732585e-01,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -1.53277087e+00],
       [ 5.03041123e-01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -1.25506714e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.92279794e+00],
       [ 3.85457285e-01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.52861770e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -8.56054006e-01],
       [ 0.00000000e+00,  4.60556404e-03,  3.63165997e-03,
        -4.92592889e-02,  0.00000000e+00, -1.03254246e+00,
         2.19857266e-19, -1.74604662e-01, -0.00000000e+00],
       [ 0.00000000e+00, -3.86803720e-02, -3.05009240e-02,
        -5.86516618e-03,  0.00000000e+00,  1.74604662e-01,
         0.00000000e+00, -1.03254246e+00,  0.00000000e+00],
       [ 0.00000000e+00, -3.07163689e-02,  3.89535929e-02,
         0.00000000e+00,  0.00000000e+00,  0.000000

In [10]:
mf.energy_elec(mf.get_init_guess())

Initial guess from minao.
MGGA ni.block_loop; input ao.shape=(10, 2348, 9), weight.shape=(2348,), coords.shape=(2348, 3)
nelec by numeric integration = 0.9997672826941552
    CPU time for vxc      0.15 sec, wall time      0.01 sec
E1 = -0.4935841095402088  Ecoul = 0.3420619878675808  Exc = -0.2869383427870689


(Array(-0.43846046, dtype=float64), Array(0.05512365, dtype=float64))

In [11]:
mf.energy_nuc()

0.0

In [12]:
e_tot, mf.conv_tol

(Array(0., dtype=float64), 1e-06)

In [13]:
def generate_network_eval_xc(mf, dm, network):
    '''
    Generates a function to overwrite eval_xc with on the mf object, for use in training with pyscfad's SCF cycle

    :param mf: Pyscfad calculation kernel object
    :type mf: Pyscfad calculation kernel object
    :param dm: Initial density matrix to use in the cycle
    :type dm: jax.Array
    :param network: The network to use in evaluating the SCF cycle
    :type network: xcquinox.xc.eXC
    :return: A function `eval_xc` that uses an xcquinox network as the pyscfad kernel calculation driver.
    :rtype: function

    The returned function:

    eval_xc(xc_code, rho, ao, gw, coords, spin=0, relativity=0, deriv=1, omega=None, verbose=None)
    The function to use as driver for a pyscf(ad) calculation, using an xcquinox network.

    This overwrites mf.eval_xc with a custom function, evaluating:

    Exc_exc, vs = jax.value_and_grad(EXC_exc_vs, has_aux=True)(jnp.concatenate([jnp.expand_dims(rho0_a,-1),
                                            jnp.expand_dims(rho0_b,-1),
                                            jnp.expand_dims(gamma_a,-1),
                                            jnp.expand_dims(gamma_ab,-1),
                                            jnp.expand_dims(gamma_b,-1),
                                            jnp.expand_dims(jnp.zeros_like(rho0_a),-1), #Dummy for laplacian
                                            jnp.expand_dims(jnp.zeros_like(rho0_a),-1), #Dummy for laplacian
                                            jnp.expand_dims(tau_a,-1),
                                            jnp.expand_dims(tau_b,-1),
                                            jnp.expand_dims(non_loc_a,-1),
                                            jnp.expand_dims(non_loc_b,-1)],axis=-1))


        :param xc_code: The XC functional code string in libxc format, but it is ignored as the network is the calculation driver
        :type xc_code: str
        :param rho: The [..., *, N] arrays (... for spin polarized), N is the number of grid points.
                    rho (*,N) ordered as (rho, grad_x, grad_y, grad_z, laplacian, tau)
                    rho (2,*,N) is [(rho_up, grad_x_up, grad_y_up, grad_z_up, laplacian_up, tau_up),
                                    (rho_down, grad_x_down, grad_y_down, grad_z_down, laplacian_down, tau_down)]
                    PySCFAD doesn't do spin-polarized grid calculations yet, so this will be unpolarized.
        :type rho: jax.Array
        :param ao: The atomic orbitals on the grid to use in the network calculation. Explcitly specified as the block loops break down the grid if memory is too low
        :type ao: jax.Array
        :param ao: The grid weights to use in the network calculation. Explcitly specified as the block loops break down the grid if memory is too low
        :type ao: jax.Array
        :param ao: The grid coordinates to use in the network calculation. Explcitly specified as the block loops break down the grid if memory is too low
        :type ao: jax.Array
        :param spin: The spin of the calculation, integer valued, polarized if non-zero, defaults to zero
        :type spin: int
        :param relativity: Integer, unused right now, defaults to zero
        :type relativity: int
        :param deriv: Unused here, defaults to 1
        :type deriv: int
        :param omega: Hybrid mixing term, unused here, defaults to None
        :type omega: float
        :param verbose: Unused here, defaults to None
        :type verbose: int
        :return: ex, vxc, fxc, kxc
                 where: ex -> exc, XC energy density on the grid
                        vxc -> (vrho, vsigma, vlapl, vtau), gradients of the exc w.r.t. the quantities given.
                        Only vrho and vtau are used, vsigma=vlapl=fxc=kxc=None.
                        vrho = vs[:, 0]+vs[:, 1]
                        vtau = vs[:, 7]+vs[:, 8]
        
        :rtype: tuple
    '''
    def eval_xc(xc_code, rho, ao, gw, coords, spin=0, relativity=0, deriv=1, omega=None, verbose=None):
        '''
        The function to use as driver for a pyscf(ad) calculation, using an xcquinox network.

        This overwrites mf.eval_xc with a custom function, evaluating:

        Exc_exc, vs = jax.value_and_grad(EXC_exc_vs, has_aux=True)(jnp.concatenate([jnp.expand_dims(rho0_a,-1),
                                                jnp.expand_dims(rho0_b,-1),
                                                jnp.expand_dims(gamma_a,-1),
                                                jnp.expand_dims(gamma_ab,-1),
                                                jnp.expand_dims(gamma_b,-1),
                                                jnp.expand_dims(jnp.zeros_like(rho0_a),-1), #Dummy for laplacian
                                                jnp.expand_dims(jnp.zeros_like(rho0_a),-1), #Dummy for laplacian
                                                jnp.expand_dims(tau_a,-1),
                                                jnp.expand_dims(tau_b,-1),
                                                jnp.expand_dims(non_loc_a,-1),
                                                jnp.expand_dims(non_loc_b,-1)],axis=-1))

        :param xc_code: The XC functional code string in libxc format, but it is ignored as the network is the calculation driver
        :type xc_code: str
        :param rho: The [..., *, N] arrays (... for spin polarized), N is the number of grid points.
                    rho (*,N) ordered as (rho, grad_x, grad_y, grad_z, laplacian, tau)
                    rho (2,*,N) is [(rho_up, grad_x_up, grad_y_up, grad_z_up, laplacian_up, tau_up),
                                    (rho_down, grad_x_down, grad_y_down, grad_z_down, laplacian_down, tau_down)]
                    PySCFAD doesn't do spin-polarized grid calculations yet, so this will be unpolarized.
        :type rho: jax.Array
        :param ao: The atomic orbitals on the grid to use in the network calculation. Explcitly specified as the block loops break down the grid if memory is too low
        :type ao: jax.Array
        :param ao: The grid weights to use in the network calculation. Explcitly specified as the block loops break down the grid if memory is too low
        :type ao: jax.Array
        :param ao: The grid coordinates to use in the network calculation. Explcitly specified as the block loops break down the grid if memory is too low
        :type ao: jax.Array
        :param spin: The spin of the calculation, integer valued, polarized if non-zero, defaults to zero
        :type spin: int
        :param relativity: Integer, unused right now, defaults to zero
        :type relativity: int
        :param deriv: Unused here, defaults to 1
        :type deriv: int
        :param omega: Hybrid mixing term, unused here, defaults to None
        :type omega: float
        :param verbose: Unused here, defaults to None
        :type verbose: int
        :return: ex, vxc, fxc, kxc
                 where: ex -> exc, XC energy density on the grid
                        vxc -> (vrho, vsigma, vlapl, vtau), gradients of the exc w.r.t. the quantities given.
                        Only vrho and vtau are used, vsigma=vlapl=fxc=kxc=None.
                        vrho = vs[:, 0]+vs[:, 1]
                        vtau = vs[:, 7]+vs[:, 8]
        
        :rtype: tuple
        '''
        # print('custom eval_xc; input rho shape: ', rho.shape)
        if len(rho.shape) == 2:
            #not spin-polarized
            rho0 = rho[0] #density
            drho = rho[1:4] #grad_x, grad_y, grad_z
            #laplacian next
            # tau = 0.5*(rho[1] + rho[2] + rho[3])
            tau = rho[-1] # tau
            
            non_loc = jnp.zeros_like(tau)
            #decompose into spin channels
            rho0_a = rho0_b = rho0*0.5
            gamma_a=gamma_b=gamma_ab= jnp.einsum('ij,ij->j',drho[:],drho[:])*0.25
            tau_a = tau_b = tau*0.5
            non_loc_a=non_loc_b = non_loc*0.5
            if network.verbose:
                print(f'decomposed shapes:\nrho0={rho0.shape}\ndrho={drho.shape}\ntau={tau.shape}\nnon_loc={non_loc.shape}')
                print(f'decomposed shapes:\ngamma_a={gamma_a.shape}\ngamma_b={gamma_b.shape}\ngamma_ab={gamma_ab.shape}')
        else:
            #spin-polarized density
            rho0_a = rho[0, 0]
            rho0_b = rho[1, 0]

            drho_a = rho[0, 1:4]
            drho_b = rho[1, 1:4]
            # jnp.einsumed density gradient
            gamma_a, gamma_b = jnp.einsum('ij,ij->j',drho_a,drho_a), jnp.einsum('ij,ij->j',drho_b,drho_b)
            gamma_ab = jnp.einsum('ij,ij->j',drho_a,drho_b)
            # Kinetic energy density
            tau_a = rho[0, -1]
            tau_b = rho[1, -1]

            non_loc_a, non_loc_b = jnp.zeros_like(tau_a), jnp.zeros_like(tau_b)
            if network.verbose:
                print(f'decomposed shapes:\nrho0(a,b)={rho0_a.shape},{rho0_b.shape}\ndrho(a,b)={drho_a.shape},{drho_b.shape}\ntau(a,b)={tau_a.shape},{tau_b.shape}\nnon_loc(a,b)={non_loc_a.shape},{non_loc_b.shape}')
                print(f'decomposed shapes:\ngamma_a={gamma_a.shape}\ngamma_b={gamma_b.shape}\ngamma_ab={gamma_ab.shape}')


        # xc-energy per unit particle
        # print(f'EVALUATING GRID MODELS; OPTIONAL PARAMETERS:')
        # try:
        #     print(f'gw.shape={gw.shape}, coor.shape={coor.shape}')
        # except:
        #     print('no externally supplied gw or coor')
        # print('eval_xc eval_grid_models call')
        
        def EXC_exc_vs(x):
            exc = network.eval_grid_models(x, mf=mf, dm=dm, ao=ao, gw=gw, coor=coords)
            Exc = jnp.sum(((rho0_a + rho0_b)*exc[:,0])*gw)
            return Exc, exc
        if network.verbose:
            print(f'eval_xc -> Exc_exc and potentials on grid via autodiff')
        v_and_g_inp = jnp.concatenate([jnp.expand_dims(rho0_a,-1),
                                                jnp.expand_dims(rho0_b,-1),
                                                jnp.expand_dims(gamma_a,-1),
                                                jnp.expand_dims(gamma_ab,-1),
                                                jnp.expand_dims(gamma_b,-1),
                                                jnp.expand_dims(jnp.zeros_like(rho0_a),-1), #Dummy for laplacian
                                                jnp.expand_dims(jnp.zeros_like(rho0_a),-1), #Dummy for laplacian
                                                jnp.expand_dims(tau_a,-1),
                                                jnp.expand_dims(tau_b,-1),
                                                jnp.expand_dims(non_loc_a,-1),
                                                jnp.expand_dims(non_loc_b,-1)],axis=-1)
        print(f'v_and_g_inp.shape={v_and_g_inp.shape}')
        Exc_exc, vs = jax.value_and_grad(EXC_exc_vs, has_aux=True)(v_and_g_inp)
        print(f'Exc_exc and vs returned: Exc = {Exc_exc[0]}, exc.shape={Exc_exc[1].shape}, vs.shape={vs.shape}')
        Exc, exc = Exc_exc
        print(f'eval_xc Exc = {Exc}')
        if jnp.sum(jnp.isnan(exc[:, 0])):
            print('NaNs detected in exc. Number of NaNs: {}'.format(jnp.sum(jnp.isnan(exc[:, 0]))))
            raise
        else:
            exc = exc[:, 0]
            
        # print('ao shape: ', ao.shape)
        # print('exc from network evaluation on grid models shape: ', exc.shape)
        # print('vs from network evaluation on grid models shape: ', vs.shape)
        # print('Exc from network evaluation on grid models shape: ', Exc)

        vgf = lambda x: network(x, ao, gw, mf=mf, coor=coords)
        mf.network = network
        mf.network_eval = vgf

        #vrho; d Exc/d rho, separate spin channels
        vrho = vs[:, 0]+vs[:, 1]
        #vtau; d Exc/d tau, separate spin channels
        vtau = vs[:, 7]+vs[:, 8]
        
        vgamma = jnp.zeros_like(vrho)
        
        vlapl = None
        
        fxc = None #second order functional derivative
        kxc = None #third order functional derivative
        if network.verbose:
            print(f'shapes: vrho={vrho.shape}, vgamma={vgamma.shape}')
        return exc, (vrho, vgamma, vlapl, vtau), fxc, kxc
    return eval_xc

In [23]:
import numpy


In [ ]:
numpy.mm

In [14]:
evxc = generate_network_eval_xc(mf=mf, dm=dms[0], network=xc)

In [15]:
mft = scf.UHF(mol)
vgf = lambda x: xc(x, ao_evals[0], gws[0], mf=mf, coor=mf.grids.coords)
mft.network = xc
mft.network_eval1 = vgf
mft.kernel()

Set gradient conv threshold to 3.16228e-05
cond(S) = 20.664786798723085
E1 = -0.4935841095402088  Ecoul = 0.17103099393379043
init E= -0.322553115606418
    CPU time for initialize scf      1.46 sec, wall time      0.14 sec


<class 'pyscfad.scf.uhf.UHF'> does not have attributes  network network_eval1
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1215: UserWarning: Function mol.dumps drops attribute coords because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1215: UserWarning: Function mol.dumps drops attribute exp because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1215: UserWarning: Function mol.dumps drops attribute ctr_coeff because it is not JSON-serializable
  warnings.warn(msg)


multiplicity <S^2> = 0.75  2S+1 = 2
_scf cycle call, generated vhf = mf.get_veff(mol, dm) = [[[ 0.27521831  0.01176717 -0.10350744  0.          0.
    0.          0.          0.          0.        ]
  [ 0.01176717  0.03264282 -0.02429322  0.          0.
    0.          0.          0.          0.        ]
  [-0.10350744 -0.02429322  0.05120986  0.          0.
    0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.51056858  0.
    0.          0.11607014  0.          0.        ]
  [ 0.          0.          0.          0.          0.51056858
    0.          0.          0.11607014  0.        ]
  [ 0.          0.          0.          0.          0.
    0.51056858  0.          0.          0.11607014]
  [ 0.          0.          0.          0.11607014  0.
    0.          0.27486981  0.          0.        ]
  [ 0.          0.          0.          0.          0.11607014
    0.          0.          0.27486981  0.        ]
  [ 0.          0.          0.      

Array(-0.70938178, dtype=float64)

In [16]:
mft.e_tot

Array(-0.70938178, dtype=float64)

In [17]:
L = jnp.eye(dm.shape[-1])
scaling = jnp.ones(dm.shape[-1]*2)

In [18]:
mft.make_rdm1()

Array([[[ 1.27400796e-01,  2.32387783e-01,  3.11460076e-02,
         -1.46803426e-09, -1.46803427e-09, -1.46803427e-09,
          2.30933712e-08,  2.30933714e-08,  2.30933713e-08],
        [ 2.32387783e-01,  4.23891241e-01,  5.68124523e-02,
         -2.67779510e-09, -2.67779511e-09, -2.67779511e-09,
          4.21238917e-08,  4.21238920e-08,  4.21238918e-08],
        [ 3.11460076e-02,  5.68124523e-02,  7.61434638e-03,
         -3.58894197e-10, -3.58894198e-10, -3.58894198e-10,
          5.64569718e-09,  5.64569723e-09,  5.64569720e-09],
        [-1.46803426e-09, -2.67779510e-09, -3.58894197e-10,
          1.69160999e-17,  1.69161000e-17,  1.69161000e-17,
         -2.66103990e-16, -2.66103992e-16, -2.66103991e-16],
        [-1.46803427e-09, -2.67779511e-09, -3.58894198e-10,
          1.69161000e-17,  1.69161001e-17,  1.69161001e-17,
         -2.66103991e-16, -2.66103994e-16, -2.66103992e-16],
        [-1.46803427e-09, -2.67779511e-09, -3.58894198e-10,
          1.69161000e-17,  1.691610

In [19]:
dm.ndim

2

In [20]:
mft.scf_summary

{'e1': Array(-0.46472867, dtype=float64),
 'e2': Array(-0.2446531, dtype=float64),
 'nuc': 0.0}

In [21]:
mft2 = scf.UHF(mol)
mft2.kernel()

Set gradient conv threshold to 3.16228e-05
cond(S) = 20.664786798723085
E1 = -0.4935841095402088  Ecoul = 0.17103099393379043
init E= -0.322553115606418
    CPU time for initialize scf      0.23 sec, wall time      0.02 sec
multiplicity <S^2> = 0.75  2S+1 = 2
E1 = -0.4906045058048447  Ecoul = 1.2379730304809626e-17
cycle= 1 E= -0.490604505804845  delta_E= -0.168  |g|= 0.0722  |ddm|= 0.406
    CPU time for cycle = 1      0.39 sec, wall time      0.02 sec
diis-norm(errvec)=0.118558
diis-c [-0.01405603  1.        ]
multiplicity <S^2> = 0.75  2S+1 = 2
E1 = -0.49843474085714373  Ecoul = 2.4805842064132175e-17
cycle= 2 E= -0.498434740857144  delta_E= -0.00783  |g|= 0.028  |ddm|= 0.146
    CPU time for cycle = 2      0.71 sec, wall time      0.05 sec
diis-norm(errvec)=0.0458698
diis-c [-2.52783350e-07 -6.30994556e-01  1.63099456e+00]
multiplicity <S^2> = 0.75  2S+1 = 2
E1 = -0.49980980605573033  Ecoul = 1.1369955874814656e-17
cycle= 3 E= -0.49980980605573  delta_E= -0.00138  |g|= 0.000172  |d

/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1215: UserWarning: Function mol.dumps drops attribute coords because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1215: UserWarning: Function mol.dumps drops attribute exp because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1215: UserWarning: Function mol.dumps drops attribute ctr_coeff because it is not JSON-serializable
  warnings.warn(msg)


multiplicity <S^2> = 0.75  2S+1 = 2
E1 = -0.49980983223188896  Ecoul = 1.1612123446046894e-17
Extra cycle  E= -0.499809832231889  delta_E= -2.22e-16  |g|= 1.39e-09  |ddm|= 6.21e-09
    CPU time for scf_cycle      3.39 sec, wall time      0.23 sec


Array(-0.49980983, dtype=float64)

In [22]:
raise

RuntimeError: No active exception to reraise

In [ ]:
mft.get_veff??

In [ ]:
from pyscf import dft as dft_pyscf

In [ ]:
mf3 = dft_pyscf.UKS(mol, xc='SCAN')
dminp = mf3.get_init_guess()
evxc2 = generate_network_eval_xc(mf=mf3, dm=dminp, network=xc)
mf3.grids.level =3
mf3.define_xc_(evxc, 'MGGA')
mf3.kernel()

In [ ]:
vgf = lambda x: xc(x, ao_evals[0], gws[0], mf=mf, coor=mf.grids.coords)
mft.network = xc
mft.network_eval1 = vgf


In [ ]:
class E_PySCFAD_loss(eqx.Module):
    def __init__(self):
        '''
        The standard energy loss module, RMSE loss of predicted vs. reference energies.
        '''
        super().__init__()

    def __call__(self, model, mf, inp_dm, ao, gw, ref_en):
        '''
        Computes the energy loss for a given model and associated input density matrix, atomic orbitals on the grid, and grid weights

        Loss is the RMSE energy, so predicted energy can potentially be a jax.Array of SCF guesses.

        :param model: The XC object whose forward pass predicts the XC energy based on the inputs here.
        :type model: xcquinox.xc.eXC
        :param inp_dm: The density matrix to pass into the network for density creation on the grid.
        :type inp_dm: jax.Array
        :param ref_en: The reference energy to take the loss with respect to.
        :type ref_en: jax.Array
        :param ao_eval: Atomic orbitals evaluated on the grid
        :type ao_eval: jax.Array
        :param grid_weights: pyscfad's grid weights for the reference calculation
        :type grid_weights: jax.Array
        :return: The RMSE error.
        :rtype: jax.Array
        '''
        print('generating eval_xc function to overwrite')
        # vgf = lambda x: model(x, ao, gw, mf=mf)
        # mf.network = model
        # mf.network_eval = vgf
        mf.max_memory=16000
        # evxc = generate_network_eval_xc(mf=mf, dm=inp_dm, ao=ao, gw=gw, network=model)
        evxc = generate_network_eval_xc(mf=mf, dm=inp_dm, network=model)
        mf.define_xc_(evxc, xctype='MGGA')
        print('predicting energy...')
        e_pred = mf.kernel()
        print('energy predicted')
        eL = jnp.sqrt( jnp.mean((e_pred-ref_en)**2))
        return eL


In [ ]:
cpus = jax.devices(backend='cpu')

In [ ]:
scheduler = optax.exponential_decay(init_value = 1e-2, transition_begin=50, transition_steps=500, decay_rate=0.9)
optimizer = optax.adam(learning_rate = 1e-2)
# optimizer = optax.adam(learning_rate = scheduler)

trainer = xce.train.xcTrainer(model=xc, optim=optimizer, steps=500, loss = E_PySCFAD_loss(), do_jit=False, logfile='log')
# with jax.default_device(cpus[0]):
#     newm = trainer(1, trainer.model, mfs, dms, ao_evals, gws, [-109.52596483])
newm = trainer(1, trainer.model, mfs, dms, ao_evals, gws, [-109.52596483])


Modifications in mldftdat, pyscfad, pyscf, xcquinox. 

xcquinox modifications took place in package repo, so no need to find differences

mldftdat and pyscfad changes occured in package directory, need to find differences for mldftdat to put into xcquinox-cider and a patch for pyscfad

In dft.rks; np.isnan check at line 57, import jax

In dft.numint, anywhere the loop generates subset ao/grids to loop over eval_xc with, tagged with #XCQUINOX MODIFICATION

In [ ]:
jax.clear_backends()

In [ ]:
jax.clear_caches()

In [ ]:
eqx.clear_caches()

In [ ]:
pyscfad.jax.clear_backends()

In [ ]:
pyscfad.jax.clear_caches()

In [ ]:
pyscfad.jax.c